# 1. Required Packages

In [7]:
import json
import joblib
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Concatenate
from tensorflow.keras.optimizers import Adam, RMSprop, SGD, Nadam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import AUC, Recall
from utils import get_data_path, get_save_path, save_json

# 2. Data Loading

In [8]:
X_train = pd.read_csv(get_data_path("X_train_deep.csv"))
y_train = pd.read_csv(get_data_path("y_train.csv")).squeeze()
X_val = pd.read_csv(get_data_path("X_val_deep.csv"))
y_val = pd.read_csv(get_data_path("y_val.csv")).squeeze()
input_dim = X_train.shape[1]

## Load best hyperparameters

In [9]:
with open(get_data_path("best_hyperparameters_dense.json")) as f:
    best_hp_dense = json.load(f)
with open(get_data_path("best_hyperparameters_widedeep.json")) as f:
    best_hp_wd = json.load(f)

# 3. Functions

## Build Final Dense Feedforward Model

In [10]:
def build_best_dense_model(hp):
    model = Sequential()
    model.add(Input(shape=(input_dim,)))
    for i in range(hp['num_layers']):
        model.add(Dense(hp[f'units_{i}'], activation=hp['activation']))
        if hp.get(f'use_bn_{i}', False):
            model.add(BatchNormalization())
        model.add(Dropout(hp[f'dropout_{i}']))
    model.add(Dense(1, activation='sigmoid'))

    lr = hp['learning_rate']
    if hp['optimizer'] == 'adam':
        optimizer = Adam(learning_rate=lr)
    elif hp['optimizer'] == 'rmsprop':
        optimizer = RMSprop(learning_rate=lr)
    elif hp['optimizer'] == 'sgd':
        optimizer = SGD(learning_rate=lr)
    else:
        optimizer = Nadam(learning_rate=lr)

    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=["accuracy", Recall(name="recall"), AUC(name="AUC")])
    return model


## Build Final Wide & Deep Model

In [11]:
def build_best_wide_and_deep_model(hp):
    input_layer = Input(shape=(input_dim,))
    deep = input_layer
    for i in range(hp['num_layers']):
        deep = Dense(hp[f'units_{i}'], activation=hp['activation'])(deep)
        if hp.get(f'use_bn_{i}', False):
            deep = BatchNormalization()(deep)
        deep = Dropout(hp[f'dropout_{i}'])(deep)

    combined = Concatenate()([input_layer, deep])
    output = Dense(1, activation='sigmoid')(combined)
    model = Model(inputs=input_layer, outputs=output)

    lr = hp['learning_rate']
    if hp['optimizer'] == 'adam':
        optimizer = Adam(learning_rate=lr)
    elif hp['optimizer'] == 'rmsprop':
        optimizer = RMSprop(learning_rate=lr)
    elif hp['optimizer'] == 'sgd':
        optimizer = SGD(learning_rate=lr)
    else:
        optimizer = Nadam(learning_rate=lr)

    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=["accuracy", Recall(name="recall"), AUC(name="AUC")])
    return model


# 4. Training Loop

In [12]:
# Configure EarlyStopping
early_stop = EarlyStopping(monitor="val_AUC", mode="max", patience=10, restore_best_weights=True)

# Train Dense model
dense_model = build_best_dense_model(best_hp_dense)
history_dense = dense_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=best_hp_dense.get("batch_size", 32),
    callbacks=[early_stop],
    verbose=2
)
dense_model.save(get_save_path("final_model_dense.keras"))

# Save Dense history
joblib.dump(history_dense.history, get_save_path("history_dense.pkl"))

# Train Wide & Deep model
wd_model = build_best_wide_and_deep_model(best_hp_wd)
history_wd = wd_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=best_hp_wd.get("batch_size", 32),
    callbacks=[early_stop],
    verbose=2
)
wd_model.save(get_save_path("final_model_widedeep.keras"))

# Save Wide & Deep history
joblib.dump(history_wd.history, get_save_path("history_widedeep.pkl"))

Epoch 1/100
958/958 - 3s - 3ms/step - AUC: 0.5418 - accuracy: 0.6828 - loss: 0.6391 - recall: 0.0964 - val_AUC: 0.6086 - val_accuracy: 0.7113 - val_loss: 0.5872 - val_recall: 0.0415
Epoch 2/100
958/958 - 1s - 1ms/step - AUC: 0.5964 - accuracy: 0.7091 - loss: 0.5918 - recall: 0.0327 - val_AUC: 0.6238 - val_accuracy: 0.7113 - val_loss: 0.5809 - val_recall: 0.0160
Epoch 3/100
958/958 - 1s - 1ms/step - AUC: 0.6172 - accuracy: 0.7120 - loss: 0.5846 - recall: 0.0333 - val_AUC: 0.6266 - val_accuracy: 0.7085 - val_loss: 0.5820 - val_recall: 0.0619
Epoch 4/100
958/958 - 1s - 2ms/step - AUC: 0.6227 - accuracy: 0.7144 - loss: 0.5819 - recall: 0.0560 - val_AUC: 0.6248 - val_accuracy: 0.7106 - val_loss: 0.5832 - val_recall: 0.0248
Epoch 5/100
958/958 - 1s - 1ms/step - AUC: 0.6319 - accuracy: 0.7126 - loss: 0.5795 - recall: 0.0476 - val_AUC: 0.6259 - val_accuracy: 0.7087 - val_loss: 0.5819 - val_recall: 0.0211
Epoch 6/100
958/958 - 1s - 1ms/step - AUC: 0.6349 - accuracy: 0.7122 - loss: 0.5780 - reca

['/Users/audreychang/projects/ACTL3143/Actl3143-report2/data/history_widedeep.pkl']